# Server Side Socketing

Taking code from https://docs.python.org/3.5/howto/sockets.html
and https://realpython.com/python-sockets/#socket-api-overview

There are two parts to sockets, the server and the client that tries to call upon the server.
The code below is for the server side. 

In [1]:
#Libraries
import socket
import fcntl, struct

In [2]:
#takes in callback function (string to string)
#calls this function on the data received once
def doonServer(callback):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) #Port is marked as reusable even if in TIME_WAIT status

##Get host ip (this sets it to '127.0.1.1' due to pi settings)
    host = socket.gethostbyname(socket.gethostname())
    print(host)

#Gets ip address tied to network adapter
    host = socket.inet_ntoa(fcntl.ioctl(s.fileno(), 0x8915, struct.pack('256s', b'wlan0'))[20:24])
    print(host)

#Takes in a tuple, First argument in tuple is the host, Second is the port number
#Port number is arbitrary but above 1024
    s.bind((host, 6544)) #loopback
    s.listen() 

#The actual socket used to interface with client socket
#Shortcoming: Only accepts one client, closes after accepting one client.
    conn, adr = s.accept()
    while True:
        data = conn.recv(1024) #number of bytes allocated for buffer to read in data
        if not data: #when no more data is received
            conn.close()
            break
    #a byte-like object is in data
    #we can do whatever we want with it
        string = data.decode("utf-8")
        conn.sendall(callback(string).encode("utf-8"))
    s.close()

In [3]:
#returns string
#Sample string to string function
def onString(str):
    if (str == "Hi" or str == "Hello"):
        return "Bye"
    else:
        return "How Unfriendly!"

In [4]:
doonServer(onString)

127.0.1.1
10.245.155.164


Using the above code and the code within Client socket, you should be able to communicate between Raspberry Pis, by setting one as a server and the other as a client, using their IP addresses.

In [21]:
# Server Code:
import socket
import fcntl, struct

# Port has to match client side socket
PORT_num = 6543 # choose a number here; arbitrary but has to be above 1024
BUFFER = 1024 # buffer size

# function to process a message coming into the server
def process_message(message_in):
    if (message_in == "Hi" or message_in == "Hello"):
        message_out = "Bye"
    else:
        message_out = "How Unfriendly!"
    return message_out

# set up and run server:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Port is marked as reusable even if in TIME_WAIT status
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# Set up host (gets IP address tied to network adapter)
host = socket.inet_ntoa(fcntl.ioctl(s.fileno(), 0x8915, struct.pack('256s', b'wlan0'))[20:24])
print('Server set up:',host)

s.bind((host, PORT_num))
s.listen()

# set up to accept incoming message (interface with client socket)
# limitation: only accepts one client, closes after accepting one client
conn, adr = s.accept()
while True:
    data = conn.recv(BUFFER) # read in data (size of buffer)
    if not data: # when no more data is received
        conn.close()
        break
    message_in = data.decode("utf-8") # data is byte-like object, decode to string
    message_out = process_message(message_in) # process
    conn.sendall(message_out.encode("utf-8")) # send reply back to client
s.close()

Server set up: 192.168.0.113
